In [2]:
!git clone https://github.com/Sandipan99/POLAR.git

Cloning into 'POLAR'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 181 (delta 78), reused 115 (delta 35), pack-reused 0
Receiving objects: 100% (181/181), 11.08 MiB | 12.15 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [1]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip


--2022-03-14 15:28:51--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-03-14 15:28:51--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  4.89MB/s    in 5m 56s  

2022-03-14 15:34:48 (5.03 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [3]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

!pip install ipython-autotime
%load_ext autotime

time: 234 µs (started: 2022-03-14 15:38:29 +00:00)


In [4]:
model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt') ## needed for glove embeddings only 


time: 33.4 s (started: 2022-03-14 15:38:52 +00:00)


In [8]:
word_list = model_glove.wv

time: 9.61 ms (started: 2022-03-14 15:49:42 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [11]:
word_lists = word_list.vocab.keys()

time: 5.38 ms (started: 2022-03-14 15:51:44 +00:00)


In [36]:
fortune500 = ['Walmart Inc. ',
 'Amazon.com, Inc. ',
 'Apple Inc. ',
 'CVS Health Corporation ',
 'UnitedHealth Group Incorporated ',
 'Berkshire Hathaway',
 'McKesson Corporation ',
 'AmerisourceBergen Corporation ',
 'Alphabet Inc. ',
 'Exxon Mobil Corporation ',
 'AT&T Inc. ',
 'Costco Wholesale Corporation ',
 'Cigna Corporation ',
 'Cardinal Health, Inc. ',
 'Microsoft Corporation ',
 'Walgreens Boots Alliance, Inc. ',
 'Kroger Company ',
 'Home Depot, Inc. ',
 'JP Morgan Chase & Co. ',
 'Verizon Communications Inc. ',
 'Ford Motor Company ',
 'General Motors Company ',
 'Anthem, Inc. ',
 'Centene Corporation ',
 'FEDERAL NATIONAL MORTGAGE ASSOC ',
 'Comcast Corporation ',
 'Chevron Corporation ',
 'Dell Technologies Inc. ',
 'Bank of America Corporation ',
 'Target Corporation ',
 "Lowe's Companies, Inc. ",
 'Marathon Petroleum Corporation ',
 'Citigroup, Inc. ',
 'Facebook, Inc. ',
 'United Parcel Service, Inc. ',
 'Johnson & Johnson ',
 'Wells Fargo & Company ',
 'General Electric Company ',
 'State Farm Insurance',
 'Intel Corporation ',
 'Humana Inc. ',
 'International Business Machines ',
 'Procter & Gamble Company ',
 'Pepsico, Inc. ',
 'FedEx Corporation ',
 'MetLife, Inc. ',
 'FEDERAL HOME LOAN MORTGAGE CORP ',
 'Phillips 66 ',
 'Lockheed Martin Corporation ',
 'Walt Disney Company ',
 'Archer-Daniels-Midland Company ',
 'Albertsons Companies, Inc. ',
 'Valero Energy Corporation ',
 'Boeing Company ',
 'Prudential Financial, Inc. ',
 'HP Inc. ',
 'Raytheon Technologies Corporati ',
 'StoneX Group Inc. ',
 'Goldman Sachs Group, Inc. ',
 'Sysco Corporation ',
 'Morgan Stanley ',
 'HCA Healthcare, Inc. ',
 'Cisco Systems, Inc. ',
 'Charter Communications, Inc. ',
 'Merck & Company, Inc. ',
 'Best Buy Co., Inc. ',
 'New York Life Insurance',
 'AbbVie Inc. ',
 'Publix Super Markets',
 'Allstate Corporation ',
 'Liberty Mutual Insurance Group',
 'AIG',
 'Tyson Foods, Inc. ',
 'Progressive Corporation ',
 'Bristol-Myers Squibb Company ',
 'Nationwide',
 'Pfizer, Inc. ',
 'Caterpillar, Inc. ',
 'TIAA',
 'Oracle Corporation ',
 'Energy Transfer LP ',
 'Dow Inc. ',
 'American Express Company ',
 'General Dynamics Corporation ',
 'Nike, Inc. ',
 'Northrop Grumman Corporation ',
 'USAA',
 'Deere & Company ',
 'Abbott Laboratories ',
 'Northwestern Mutual',
 'Dollar General Corporation ',
 'Exelon Corporation ',
 'Coca-Cola Company ',
 'Honeywell International Inc. ',
 'Thermo Fisher Scientific Inc ',
 '3M Company ',
 'TJX Companies, Inc. ',
 'The Travelers Companies, Inc. ',
 'Capital One Financial Corporati ',
 'Tesla, Inc. ',
 'Philip Morris International Inc ',
 'Arrow Electronics, Inc. ',
 'CHS',
 'Jabil Inc. ',
 'Enterprise Products Partners L. ',
 'Hewlett Packard Enterprise Comp ',
 'United Natural Foods, Inc. ',
 'Mondelez International, Inc. ',
 'ViacomCBS Inc. ',
 'The Kraft Heinz Company ',
 'Dollar Tree, Inc. ',
 'Amgen Inc. ',
 'U.S. Bancorp ',
 'Performance Food Group Company ',
 'Netflix, Inc. ',
 'Gilead Sciences, Inc. ',
 'Synnex Corporation ',
 'Eli Lilly and Company ',
 'Truist Financial Corporation ',
 'PNC Financial Services Group, I ',
 'Broadcom Inc. ',
 'CBRE Group Inc ',
 'Massachusetts Mutual Life Insurance',
 'QUALCOMM Incorporated ',
 'Starbucks Corporation ',
 'Duke Energy Corporation ',
 'Plains GP Holdings, L.P. ',
 'US Foods Holding Corp. ',
 'Lennar Corporation ',
 'Danaher Corporation ',
 'AFLAC Incorporated ',
 'Rite Aid Corporation ',
 'Visa Inc. ',
 'PayPal Holdings, Inc. ',
 'Micron Technology, Inc. ',
 'CarMax Inc ',
 'Salesforce.com Inc ',
 'Altria Group, Inc. ',
 'Lumen Technologies, Inc. ',
 'Baker Hughes Company ',
 'International Paper Company ',
 'Hartford Financial Services Gro ',
 'Penske Automotive Group, Inc. ',
 'DuPont de Nemours, Inc. ',
 'AutoNation, Inc. ',
 'Southern Company ',
 'World Fuel Services Corporation ',
 'D.R. Horton, Inc. ',
 'Nucor Corporation ',
 'Cummins Inc. ',
 'NGL ENERGY PARTNERS LP ',
 'DXC Technology Company ',
 'Union Pacific Corporation ',
 'Whirlpool Corporation ',
 'Molina Healthcare Inc ',
 'ConocoPhillips ',
 "McDonald's Corporation ",
 'Kimberly-Clark Corporation ',
 'PACCAR Inc. ',
 'PG&E',
 'CDW Corporation ',
 'Sherwin-Williams Company ',
 'L3Harris Technologies, Inc. ',
 'M&T Bank Corporation ',
 'ManpowerGroup ',
 'NextEra Energy, Inc. ',
 'Tenet Healthcare Corporation ',
 'Avnet, Inc. ',
 'General Mills, Inc. ',
 'Westrock Company ',
 'Carrier Global Corporation ',
 'Lincoln National Corporation ',
 'Genuine Parts Company ',
 'American Airlines Group, Inc. ',
 'Marsh & McLennan Companies, Inc ',
 'Applied Materials, Inc. ',
 'Becton, Dickinson and Company ',
 'Delta Air Lines, Inc. ',
 'Lear Corporation ',
 'The Bank of New York Mellon Cor ',
 'Emerson Electric Company ',
 'Western Digital Corporation ',
 'Occidental Petroleum Corporatio ',
 'NVIDIA Corporation ',
 'Cognizant Technology Solutions ',
 'Jones Lang LaSalle Incorporated ',
 'Synchrony Financial ',
 'Colgate-Palmolive Company ',
 'AECOM ',
 'XPO Logistics, Inc. ',
 'C.H. Robinson Worldwide, Inc. ',
 'BlackRock, Inc. ',
 'Dominion Energy, Inc. ',
 'Rocket Companies, Inc. ',
 "Kohl's Corporation ",
 'Fluor Corporation ',
 'DISH Network Corporation ',
 "BJ's Wholesale Club Holdings, I ",
 'Tenneco Inc. ',
 'United Airlines Holdings, Inc. ',
 'Mastercard Incorporated ',
 'Waste Management, Inc. ',
 'PBF Energy Inc. ',
 'American Electric Power Company ',
 'Fiserv, Inc. ',
 'Principal Financial Group Inc ',
 'Reinsurance Group of America, I ',
 'Automatic Data Processing, Inc. ',
 'Stanley Black & Decker, Inc. ',
 'Texas Instruments Incorporated ',
 'Halliburton Company ',
 'Stryker Corporation ',
 'Estee Lauder Companies, Inc. ',
 'Corteva, Inc. ',
 'Freeport-McMoRan, Inc. ',
 'Qurate Retail',
 'Wayfair Inc. ',
 'Laboratory Corporation of Ameri ',
 "Land O'Lakes",
 'PPG Industries, Inc. ',
 'Gap, Inc. ',
 'Kellogg Company ',
 'Parker-Hannifin Corporation ',
 'Core-Mark Holding Company, Inc. ',
 'Jacobs Engineering Group Inc. ',
 'Edison International ',
 'Guardian Life Ins. Co. of America',
 'Biogen Inc. ',
 'Omnicom Group Inc. ',
 'Unum Group ',
 'Lithia Motors, Inc. ',
 'American Financial Group, Inc. ',
 'Discover Financial Services ',
 'Adobe Inc. ',
 'Aramark ',
 'Otis Worldwide Corporation ',
 'Ecolab Inc. ',
 'AutoZone, Inc. ',
 'Loews Corporation ',
 'Illinois Tool Works Inc. ',
 'Fidelity National Information S ',
 'Ross Stores, Inc. ',
 "Peter Kiewit Sons'",
 'Equitable Holdings, Inc. ',
 'WESCO International, Inc. ',
 'The Goodyear Tire & Rubber Comp ',
 'Fox Corporation ',
 'Leidos Holdings, Inc. ',
 'Consolidated Edison, Inc. ',
 'DTE Energy Company ',
 'Charles Schwab Corporation ',
 'State Street Corporation ',
 'Ameriprise Financial, Inc. ',
 'Viatris Inc. ',
 'Sempra Energy ',
 'Farmers Insurance Exchange',
 'L Brands, Inc. ',
 'W.W. Grainger, Inc. ',
 'Community Health Systems, Inc. ',
 'Ball Corporation ',
 'Berry Global Group, Inc. ',
 'Kinder Morgan, Inc. ',
 'V.F. Corporation ',
 'Baxter International Inc. ',
 'Textron Inc. ',
 'LKQ Corporation ',
 'Keurig Dr Pepper Inc. ',
 "O'Reilly Automotive, Inc. ",
 'Crown Holdings, Inc. ',
 'Universal Health Services, Inc. ',
 'DaVita Inc. ',
 'Xcel Energy Inc. ',
 'Newmont Corporation ',
 'Vistra Corp. ',
 'IQVIA Holdings, Inc. ',
 'eBay Inc. ',
 'Corning Incorporated ',
 'Quanta Services, Inc. ',
 'HollyFrontier Corporation ',
 'Bed Bath & Beyond Inc. ',
 'Uber Technologies, Inc. ',
 'Mutual of Omaha Insurance',
 'ConAgra Brands, Inc. ',
 'PulteGroup, Inc. ',
 'EOG Resources, Inc. ',
 'Group 1 Automotive, Inc. ',
 'Ally Financial Inc. ',
 'FNF Group of Fidelity National ',
 'Nordstrom, Inc. ',
 'Discovery, Inc. - Series A ',
 'Tractor Supply Company ',
 'CSX Corporation ',
 'Marriott International ',
 'FirstEnergy Corp. ',
 'BorgWarner Inc. ',
 'Jones Financial (Edward Jones)',
 'Republic Services, Inc. ',
 'Henry Schein, Inc. ',
 'Expeditors International of Was ',
 'Entergy Corporation ',
 'Advance Auto Parts Inc Advance ',
 'Assurant, Inc. ',
 'Pacific Gas & Electric Co. ',
 'Lam Research Corporation ',
 'Boston Scientific Corporation ',
 'Altice USA, Inc. ',
 'Norfolk Southern Corporation ',
 'Sonic Automotive, Inc. ',
 'Advanced Micro Devices, Inc. ',
 'United States Steel Corporation ',
 'Markel Corporation ',
 'The ODP Corporation ',
 'The AES Corporation ',
 'Molson Coors Beverage Company ',
 'J.B. Hunt Transport Services, I ',
 'KKR & Co. Inc. ',
 'Hormel Foods Corporation ',
 'Public Service Enterprise Group ',
 'Steel Dynamics, Inc. ',
 "Dick's Sporting Goods Inc ",
 'Mohawk Industries, Inc. ',
 'Murphy USA Inc. ',
 'Square, Inc. ',
 'Quest Diagnostics Incorporated ',
 'Newell Brands Inc. ',
 'Liberty Interactive Corporation ',
 'Huntington Ingalls Industries, ',
 'Cheniere Energy, Inc. ',
 'SpartanNash Company ',
 'Alcoa Corporation ',
 'AGCO Corporation ',
 'Voya Financial, Inc. ',
 'NRG Energy, Inc. ',
 'Interpublic Group of Companies, ',
 'Campbell Soup Company ',
 'Southwest Airlines Company ',
 'News Corporation ',
 'Eversource Energy ',
 'Alleghany Corporation ',
 'Air Products and Chemicals, Inc ',
 'Auto-Owners Insurance',
 'CenterPoint Energy, Inc ',
 'Reliance Steel & Aluminum Co. ',
 'EMCOR Group, Inc. ',
 'Owens & Minor, Inc. ',
 'Mosaic Company ',
 'Erie Insurance Group',
 'Genworth Financial Inc ',
 'Amphenol Corporation ',
 'Builders FirstSource, Inc. ',
 'ONEOK, Inc. ',
 'United Rentals, Inc. ',
 'Brighthouse Financial, Inc. ',
 'Regeneron Pharmaceuticals, Inc. ',
 'Eastman Chemical Company ',
 'CommScope Holding Company, Inc. ',
 'Ryder System, Inc. ',
 'Fifth Third Bancorp ',
 'Constellation Brands, Inc. ',
 'Insight Enterprises, Inc. ',
 'Global Partners LP Global Partn ',
 'Univar Solutions Inc. ',
 'Yum China Holdings, Inc. ',
 'Targa Resources, Inc. ',
 'Intercontinental Exchange Inc. ',
 'The Andersons, Inc. ',
 'Raymond James Financial, Inc. ',
 'Thor Industries, Inc. ',
 'Thrivent Financial for Lutherans',
 'The Hershey Company ',
 'Caseys General Stores, Inc. ',
 'W.R. Berkley Corporation ',
 'Activision Blizzard, Inc ',
 'Western & Southern Financial Group',
 'American Tower Corporation ',
 'American International Group, I ',
 'Darden Restaurants, Inc. ',
 'J.M. Smucker Company ',
 'Williams Companies, Inc. ',
 'Intuit Inc. ',
 'Citizens Financial Group, Inc. ',
 'PPL Corporation ',
 'NVR, Inc. ',
 'Westinghouse Air Brake Technolo ',
 'Foot Locker, Inc. ',
 'Cincinnati Financial Corporatio ',
 'Weyerhaeuser Company ',
 'Westlake Chemical Corporation ',
 'Navistar International Corporat ',
 "Macy's Inc ",
 'Booz Allen Hamilton Holding Cor ',
 'Autoliv, Inc. ',
 'S&P Global Inc. ',
 'Global Payments Inc. ',
 'Motorola Solutions, Inc. ',
 'KeyCorp ',
 'Delek US Holdings, Inc. ',
 'Masco Corporation ',
 'Graybar Electric',
 'WEC Energy Group, Inc. ',
 'Old Republic International Corp ',
 'Frontier Communications Parent, ',
 'Chewy, Inc. ',
 'PVH Corp. ',
 'Asbury Automotive Group Inc ',
 'Seaboard Corporation ',
 'Polaris Inc. ',
 'Dana Incorporated ',
 'First American Corporation ',
 'Cintas Corporation ',
 'Toll Brothers, Inc. ',
 'SCIENCE APPLICATIONS INTERNATIO ',
 'Owens Corning Inc ',
 'Zimmer Biomet Holdings, Inc. ',
 'Xerox Holdings Corporation ',
 'Arthur J. Gallagher & Co. ',
 'Avery Dennison Corporation ',
 'Sanmina Corporation ',
 'Jefferies Financial Group Inc. ',
 'Beacon Roofing Supply, Inc. ',
 'Select Medical Holdings Corpora ',
 'Oshkosh Corporation ',
 'FM Global',
 'Booking Holdings Inc. Common St ',
 'Williams-Sonoma, Inc. ',
 'Coty Inc. ',
 'Clorox Company ',
 'Pioneer Natural Resources Compa ',
 'Dover Corporation ',
 'CMS Energy Corporation ',
 'Zoetis Inc. ',
 'Hanesbrands Inc. ',
 'Packaging Corporation of Americ ',
 'Regions Financial Corporation ',
 'Graphic Packaging Holding Compa ',
 'UGI Corporation ',
 'Sprouts Farmers Market, Inc. ',
 'Avantor, Inc. ',
 'Veritiv Corporation ',
 'Rockwell Automation, Inc. ',
 'MasTec, Inc. ',
 'DCP Midstream, LP ',
 'Northern Trust Corporation ',
 'M/I Homes, Inc. ',
 'Realogy Holdings Corp. ',
 'NCR Corporation ',
 'T. Rowe Price Group, Inc. ',
 'Vertex Pharmaceuticals Incorpor ',
 'Big Lots, Inc. ',
 'Ralph Lauren Corporation ',
 'Ulta Beauty, Inc. ',
 'Taylor Morrison Home Corporatio ',
 'Icahn Enterprises L.P. - Deposi ',
 'The Blackstone Group Inc. ',
 'O-I Glass, Inc. ',
 'Fortune Brands Home & Security, ',
 'NOV Inc. ',
 'Ovintiv Inc. ',
 'Alexion Pharmaceuticals, Inc. ',
 'Huntsman Corporation ',
 'Equinix, Inc. ',
 'ABM Industries Incorporated ',
 'Ingredion Incorporated ',
 'Chipotle Mexican Grill, Inc. ',
 'Sinclair Broadcast Group, Inc. ',
 'LPL Financial Holdings Inc. ',
 'Crown Castle International Corp ',
 'KLA Corporation ',
 'Ameren Corporation ',
 'KBR, Inc. ',
 'Burlington Stores, Inc. ',
 'Olin Corporation ',
 'CACI International, Inc. ',
 'Post Holdings, Inc. ',
 'Academy Sports and Outdoors, In ',
 'Arconic Corporation ',
 'Celanese Corporation Celanese C ',
 'Yum! Brands, Inc. ',
 'Fastenal Company ',
 'Nasdaq, Inc. ',
 'Analog Devices, Inc. ',
 'McCormick & Company, Incorporat ',
 'Carvana Co. ',
 'Franklin Resources, Inc. ',
 'Electronic Arts Inc. ',
 'MDU Resources Group, Inc. ',
 'Selective Insurance Group, Inc. ',
 'Roper Technologies, Inc. ',
 'RPM International Inc. ',
 'Cerner Corporation ',
 'Patterson Companies, Inc. ',
 'Commercial Metals Company ',
 'Boise Cascade, L.L.C. ',
 'Hasbro, Inc. ',
 'A-Mark Precious Metals, Inc. ',
 'Camping World Holdings, Inc. ',
 'NetApp, Inc. ',
 'Avis Budget Group, Inc. ',
 'R.R. Donnelley & Sons Company ',
 "Moody's Corporation "]

time: 1.24 s (started: 2022-03-14 16:03:49 +00:00)


In [39]:
fortune500_first_word = [x.split(' ')[0].lower() for x in fortune500]

time: 1.84 ms (started: 2022-03-14 16:06:42 +00:00)


['walmart',
 'amazon.com,',
 'apple',
 'cvs',
 'unitedhealth',
 'berkshire',
 'mckesson',
 'amerisourcebergen',
 'alphabet',
 'exxon',
 'at&t',
 'costco',
 'cigna',
 'cardinal',
 'microsoft',
 'walgreens',
 'kroger',
 'home',
 'jp',
 'verizon']

time: 11.3 ms (started: 2022-03-14 16:10:45 +00:00)


In [43]:
total = 0
for i in fortune500_first_word:
  if i in word_lists:
    total += 1
total

443

time: 11.3 ms (started: 2022-03-14 16:08:15 +00:00)


In [35]:
"broadcom" in word_lists

True

time: 9.02 ms (started: 2022-03-14 16:03:06 +00:00)


In [7]:
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False)

time: 5min 38s (started: 2022-03-14 15:43:05 +00:00)


In [47]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

time: 8.42 ms (started: 2022-03-14 16:15:16 +00:00)


In [49]:
generate_norm_embedding(model_glove,'glove_norm_300.mod')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1917494 [00:00<?, ?it/s]

time: 47.4 s (started: 2022-03-14 16:17:37 +00:00)


In [50]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True)

time: 33.3 s (started: 2022-03-14 16:18:30 +00:00)


In [51]:
current_model = model_gn 


time: 1.18 ms (started: 2022-03-14 16:19:11 +00:00)


In [53]:
list_antonym = []

%cd POLAR

with open('Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

/content/POLAR
time: 158 ms (started: 2022-03-14 16:20:24 +00:00)


In [54]:
list_antonym

[('implicate', 'exclude'),
 ('assertion', 'rejection'),
 ('terminal', 'beginning'),
 ('lop', 'secure'),
 ('pastor', 'libertine'),
 ('pocket', 'hook'),
 ('orbital', 'stationary'),
 ('evidence', 'disprove'),
 ('psychological', 'biological'),
 ('preparedness', 'unprepared'),
 ('saturation', 'dehydration'),
 ('geologic', 'biologic'),
 ('accessibility', 'secured'),
 ('fight', 'agree'),
 ('misconstrue', 'beautify'),
 ('lump', 'smooth'),
 ('double', 'half'),
 ('training', 'forgetting'),
 ('determined', 'negligent'),
 ('velocity', 'still'),
 ('shield', 'vulnerable'),
 ('measurement', 'guess'),
 ('iconic', 'ordinary'),
 ('thump', 'caress'),
 ('stretcher', 'compressor'),
 ('allude', 'distract'),
 ('cemetery', 'birthplace'),
 ('corrupt', 'sanctify'),
 ('cameraman', 'actor'),
 ('knowledgeable', 'uninformed'),
 ('screech', 'talk'),
 ('hedge', 'squander'),
 ('aggregate', 'please'),
 ('constrict', 'open'),
 ('make', 'break'),
 ('normative', 'descriptive'),
 ('inscribe', 'polish'),
 ('bulge', 'hollow'

time: 41.9 ms (started: 2022-03-14 16:20:33 +00:00)


In [55]:
similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3110 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1468 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1468 [00:00<?, ?it/s]

1468
time: 4.29 s (started: 2022-03-14 16:21:42 +00:00)


### Decide on the size of the antonym vector


In [56]:
num_antonym = 1468


time: 1.28 ms (started: 2022-03-14 16:23:12 +00:00)


In [57]:
## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)
time: 33.1 ms (started: 2022-03-14 16:23:14 +00:00)


In [58]:
import random

from scipy.spatial.distance import cosine as scipy_cosine
random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = defaultdict(dict)
for index_1, each_dim1 in enumerate(tqdm(t1)):
    for index_2, each_dim2 in enumerate(t1):
        dimension_similarity_matrix[index_1][index_2] = abs(1-scipy_cosine(each_dim1, each_dim2))


def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    
    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
#             print(each_dim)
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        print(test_count,min_dim)
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
#         print(working_list.shape, len(final_list))
    return final_position_index



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1468 [00:00<?, ?it/s]

time: 2min 46s (started: 2022-03-14 16:23:52 +00:00)
